In [9]:
import dis
from functools import wraps
import inspect

funcWeb = {}    # The structure that links all funcitons together
allFuncNamesInWeb = set()     # Names of all functions in funcWeb
wrapDict = {}   # key is the orginal function name.  Value is the wrapped function.
callStack = []

def autoTrigger(func):
    @wraps(func)
    def inner(*args, **kwargs):
        global funcWeb, allFuncNamesInWeb, wrapDict, callStack
        fname = func.__name__
        caller = callStack[-1] if callStack != [] else inspect.stack()[1][3]   # Who call me?
        print(f"{fname} called by {caller}")

        callStack+=[fname]

        calledFromTop = len(funcWeb[callStack[0]]['UPPER_FUNCS']) == 0
        calledFromOutside  = caller not in allFuncNamesInWeb
        
        if calledFromOutside and len(funcWeb[fname]['LOWER_FUNCS']) != 0 and len(funcWeb[fname]['UPPER_FUNCS']) != 0:
            print(f"{fname} must be at the top or bottom of structure to be called from outside.")
            ret=None
        else:    
            if caller in funcWeb[fname]['LOWER_FUNCS'] or calledFromOutside:
                print(f"called from {'outside' if calledFromOutside else 'below'}")
    
                ret = func(*args, **kwargs)
                
                if args == () and kwargs == {}:
                    try:
                        if funcWeb[fname]['RET_VALUE_NO_ARG'] == ret:
                            if not calledFromOutside: ret = None
                        else: 
                            funcWeb[fname]['RET_VALUE_NO_ARG'] = ret
                    except:
                        funcWeb[fname]['RET_VALUE_NO_ARG'] = ret
                else:
                    try:
                        if funcWeb[fname]['RET_VALUES'][[args, kwargs]] == ret:
                            if not calledFromOutside: ret = None
                        else: 
                            funcWeb[fname]['RET_VALUES'][[args, kwargs]] = ret
                    except:
                        funcWeb[fname]['RET_VALUES'][[args, kwargs]] = ret
                
                if ret is not None:    
                    for f in funcWeb[fname]['UPPER_FUNCS']:  # propagate upward
                        print(f"{fname} calls {f}  in UPPER_FUNCS loop")
                        wrapDict[f](*args, **kwargs)
    
            else: # if caller in funcWeb[fname]['UPPER_FUNCS']:: # caller is in funcWeb[fname]['UPPER_FUNCS']:
                print('called from above')
                
                if args == () and kwargs == {}:
                    if 'RET_VALUE_NO_ARG' not in funcWeb[fname] or calledFromTop:
                        ret = func(*args, **kwargs)   #  run it and store result
                        funcWeb[fname]['RET_VALUE_NO_ARG'] = ret                        
                    else:   # if there is result already
                        ret = funcWeb[fname]['RET_VALUE_NO_ARG']
                else:
                    if [args, kwargs] not in funcWeb[fname]['RET_VALUES'] or calledFromTop:
                        ret = func(*args, **kwargs)
                        funcWeb[fname]['RET_VALUES'][[args, kwargs]] = ret
                        #### funcWeb[fname]['RET_VALUES'] can get very big, so some trimming can be applied here
                    else:
                        ret = funcWeb[fname]['RET_VALUES'][[args, kwargs]]

        del callStack[-1]
        return ret

    # Put the new function at proper spot in the structure
    global funcWeb, allFuncNamesInWeb, wrapDict
    fname = func.__name__  # name of the function
    lowerFuncs = []
    upperFuncs = []
    s = set([instr.argval for instr in [instr for instr in  dis.Bytecode(func)] if instr.opname == 'LOAD_GLOBAL'])  # all functions <func> calls
#     print(inner.__name__)
    for key, value in funcWeb.items():  # Is this function <key> in the list?
        if key in s:
            value['UPPER_FUNCS'] += [fname]  # <key> will propagate up to <func>
            
    for key, value in funcWeb.items():  # does <key> call <func>
        if fname in value['FUNCS_INSIDE']:
#     print(f'pick up one  {key} ')
            upperFuncs += [key]   # <func> will propagate to <key>
            
    for key in funcWeb:        # does <func> call <key>
        if key in s:
            lowerFuncs += [key]   # record it

    for key, value in funcWeb.items():   # any other <key> that calls <func>?
        if fname in value['FUNCS_INSIDE']:
#     print(f'pick up one  {key} ')
            value['LOWER_FUNCS'] += [fname]
            
    funcWeb[fname] = {'FUNCS_INSIDE': s, 'UPPER_FUNCS': upperFuncs, 'LOWER_FUNCS':lowerFuncs, 'RET_VALUES':{}}  # <func> is new member in funcWeb
    
    allFuncNamesInWeb = allFuncNamesInWeb.union({fname})
    wrapDict[fname]=inner

    return inner


source1 = '<src1>'
source2 = '<src2>'

@autoTrigger
def processor():
    print('----- processor')
    x = producer1()
    y = producer2()
    if x is None or y is None: return None
    return x*2 + y*2

@autoTrigger
def producer1():
    global source1
    print('----- producer1')
    return source1

@autoTrigger
def consumer():
    print('----- consumer')
    x = processor()
    return x*2 if x else x

@autoTrigger
def producer2():
    global source2
    print('----- producer2')
    return source2


In [10]:
# The funcWeb is pre empty beacuse consumer, producer1 and producer2 have not been called.
funcWeb

{'processor': {'FUNCS_INSIDE': {'print', 'producer1', 'producer2'},
  'UPPER_FUNCS': ['consumer'],
  'LOWER_FUNCS': ['producer1', 'producer2'],
  'RET_VALUES': {}},
 'producer1': {'FUNCS_INSIDE': {'print', 'source1'},
  'UPPER_FUNCS': ['processor'],
  'LOWER_FUNCS': [],
  'RET_VALUES': {}},
 'consumer': {'FUNCS_INSIDE': {'print', 'processor'},
  'UPPER_FUNCS': [],
  'LOWER_FUNCS': ['processor'],
  'RET_VALUES': {}},
 'producer2': {'FUNCS_INSIDE': {'print', 'source2'},
  'UPPER_FUNCS': ['processor'],
  'LOWER_FUNCS': [],
  'RET_VALUES': {}}}

In [11]:
producer1()
funcWeb
# producer1() has its return value stored in funcWeb before processor is called
# processor calls both producers.  producer1 is not called again but producer2 is called because it has never been called.

producer1 called by <module>
called from outside
----- producer1
producer1 calls processor  in UPPER_FUNCS loop
processor called by producer1
called from below
----- processor
producer1 called by processor
called from above
producer2 called by processor
called from above
----- producer2
processor calls consumer  in UPPER_FUNCS loop
consumer called by processor
called from below
----- consumer
processor called by consumer
called from above


{'processor': {'FUNCS_INSIDE': {'print', 'producer1', 'producer2'},
  'UPPER_FUNCS': ['consumer'],
  'LOWER_FUNCS': ['producer1', 'producer2'],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src1><src1><src2><src2>'},
 'producer1': {'FUNCS_INSIDE': {'print', 'source1'},
  'UPPER_FUNCS': ['processor'],
  'LOWER_FUNCS': [],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src1>'},
 'consumer': {'FUNCS_INSIDE': {'print', 'processor'},
  'UPPER_FUNCS': [],
  'LOWER_FUNCS': ['processor'],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src1><src1><src2><src2><src1><src1><src2><src2>'},
 'producer2': {'FUNCS_INSIDE': {'print', 'source2'},
  'UPPER_FUNCS': ['processor'],
  'LOWER_FUNCS': [],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src2>'}}

In [13]:
source1 = '<src11>'
source2 = '<src22>'
producer1()
funcWeb
# producer1() has its new return value stored in funcWeb before processor is called
# processor calls both producers.  producer1 is not called again but producer2 is not called either because t has old value stored in funcWeb
# if both producers must craete new valyes, something needs to be done at processor to sync them up

producer1 called by <module>
called from outside
----- producer1
producer1 calls processor  in UPPER_FUNCS loop
processor called by producer1
called from below
----- processor
producer1 called by processor
called from above
producer2 called by processor
called from above
processor calls consumer  in UPPER_FUNCS loop
consumer called by processor
called from below
----- consumer
processor called by consumer
called from above


{'processor': {'FUNCS_INSIDE': {'print', 'producer1', 'producer2'},
  'UPPER_FUNCS': ['consumer'],
  'LOWER_FUNCS': ['producer1', 'producer2'],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src11><src11><src2><src2>'},
 'producer1': {'FUNCS_INSIDE': {'print', 'source1'},
  'UPPER_FUNCS': ['processor'],
  'LOWER_FUNCS': [],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src11>'},
 'consumer': {'FUNCS_INSIDE': {'print', 'processor'},
  'UPPER_FUNCS': [],
  'LOWER_FUNCS': ['processor'],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src11><src11><src2><src2><src11><src11><src2><src2>'},
 'producer2': {'FUNCS_INSIDE': {'print', 'source2'},
  'UPPER_FUNCS': ['processor'],
  'LOWER_FUNCS': [],
  'RET_VALUES': {},
  'RET_VALUE_NO_ARG': '<src2>'}}